In [1]:
import pandas as pd
df = pd.read_excel(r"D:\Proyectos\winter_camp\data_anot_sdc.xlsx")
df.dropna(inplace=True)
df

C:\Users\PC\AppData\Local\Temp\ipykernel_5900\1530326417.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Número de Resolución,Sección Resolutiva,Etiqueta
0,49-2016-SDC,denunciadapudoconocercuáleraelmediopublicitari...,Resolución: (149-2015/CCD)\n\nAnálisis:\n\nLeg...
1,695-2014-SC1,-2012/SC1 -INDECOPI del 9 de octubre de 2012. ...,Resolución: (004-2014/INDECOPI-JUN)\n\nAnálisi...
2,167-2015-SC1,mientras no cuenten con evidencia en contrario...,Resolución: (178-2014/CCD-INDECOPI)\n\nAnálisi...
3,301-2017-SDC,de grasa por cada porción. 40. Habiendo delimi...,Resolución: (060-2015/CCD-INDECOPI)\n\nAnálisi...
4,155-2020-SDC,constituye un factor atenuante de la conducta ...,Resolución: (367-2019/INDECOPI-AQP)\n\nAnálisi...
...,...,...,...
72,774-2014-SC1,RESOLUCIÓN 0774 -2014/SDC -INDECOPI EXPEDIENTE...,Resolución: (054-2014/CCD-INDECOPI)\n\nAnálisi...
73,139-2019-SDC,"cualquier tercero realizar, sin su consentimie...",Resolución: (208-2018/CCD-INDECOPI)\n\nAnálisi...
74,605-2014-SC1,explota ción indebida de la reputación ajena. ...,Resolución: (014-2013/CCD-INDECOPI)\n\nAnálisi...
75,108-2020-SDC,si bien en dicho do cumento se consignó a la d...,Resolución: (179-2019/CCD-INDECOPI)\n\nAnálisi...


In [2]:
def convert_to_custom_format(dataset):
    converted_data = []
    for _, row in dataset.iterrows():
        json_dict = {
            "messages": [
                {"role": "system", "content": "Vas a determinar el sentido de fallo de una resolución de la Sala de Defensa de la Competencia del Indecopi, las cuales se encuentran siempre al final del texto. El resto de texto puede servir para identificar información relativa a las infracciones evaluadas. Tu análisis consiste en dos partes: (i) identificar la resolución objeto de impugnación y (ii) identificar la infracción con el respectivo pronunciamiento. Las infracciones a la normativa de competencia desleal son: cláusula general, engaño, confusión, reputación ajena, denigración, equiparación indebida, secretos empresariales, violación de normas, sabotaje, autenticidad, legalidad, adecuación social y procedimental. El pronunciamiento debe ser calificado como 'Confirma', cuando se confirma la existencia de infracción o inexistencia de infracción. El pronunciamiento debe ser calificado como 'Confirmación parcial', cuando se admite la existencia de infracción, pero se modifica el monto de la multa a imponer. El pronunciamiento debe ser calificado como 'No confirma' cuando se pronuncia de forma negativa sobre la existencia de una infracción, para ello se ocupan términos como revocar o declarar la nulidad. Finalmente, si no hayas la sección resolutiva o el texto no versa sobre una apelación relativa a una infracción debes catalogarla como irrelevante. La respuesta debe ser devuelta de la siguiente manera: Resolución: (número de resolución o secretaría técnica), Análisis: infracción: (Confirma o Confirmación parcial o No confirma)."},
                {"role": "user", "content": row['Sección Resolutiva']},
                {"role": "assistant", "content": row['Etiqueta']}
            ]
        }
        converted_data.append(json_dict)
    return converted_data

converted_data = convert_to_custom_format(df)

In [3]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    random_state=42  # for reproducibility
)

In [4]:
import json
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)